In [ ]:
import MySQLdb
from pandas.io import sql
import pandas as pd
import numpy as np
conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor()

In [ ]:
df_encounter_diagnosis = pd.read_sql('select * from `encounter_diagnosis`', con = conn)
df_encounter_diagnosis

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()

In [ ]:
# Diagnosis Observations

In [ ]:
obs_dx_mimic = pd.read_sql('SELECT * FROM diagnoses_icd', con = mimicconn) 
obs_dx_mimic

In [ ]:
diagnoses_mimic_ = obs_dx_mimic[obs_dx_mimic['seq_num'] == 1]
diagnoses_mimic_

In [ ]:
d_icd = pd.read_sql('select * from d_icd_diagnoses', con = mimicconn)
d_icd

In [ ]:
dict_ = dict(zip(d_icd.icd_code, d_icd.long_title))
dict_

In [ ]:
diagnoses_mimic_['long_title'] = diagnoses_mimic_['icd_code'].map(dict_)
diagnoses_mimic_

In [ ]:
diagnosis_concept = pd.read_sql('select * from `concept` where class_id = 4', con = conn) 
diagnosis_concept

In [ ]:
dict_diag = dict(zip(diagnosis_concept.description, diagnosis_concept.concept_id))
dict_diag

In [ ]:
diagnoses_mimic_['concept_id'] = diagnoses_mimic_['long_title'].map(dict_diag)
diagnoses_mimic_

In [ ]:
diagnoses_mimic_.isnull().sum()

In [ ]:
diagnoses_mimic_ = diagnoses_mimic_.rename({"subject_id": "patient_id", "hadm_id": "encounter_id", "icd_code":"diagnosis_coded_name", "concept_id":"diagnosis_coded"}, axis='columns')
diagnoses_mimic_ = diagnoses_mimic_.drop(columns=['seq_num', 'icd_version', 'long_title'])
diagnoses_mimic_['date_created']= '2022-01-01'
import uuid
diagnoses_mimic_['uuid'] = [uuid.uuid4() for x in range(len(diagnoses_mimic_))]
diagnoses_mimic_['creator'] = 1


In [ ]:
diagnoses_mimic_.isnull().sum()

In [ ]:
column_names = ['diagnosis_coded','diagnosis_coded_name', 'encounter_id', 'patient_id', 'uuid', 'creator', 'date_created']

diagnoses_mimic_ = diagnoses_mimic_.reindex(columns=column_names)
diagnoses_mimic_['certainty'] = 'CONFIRMED'
diagnoses_mimic_['diagnosis_coded']= diagnoses_mimic_['diagnosis_coded'].astype(int, errors='ignore')
diagnoses_mimic_= diagnoses_mimic_.replace({np.NAN: None})
diagnoses_mimic_


In [ ]:
# Procedures observations

In [ ]:
obs_proc_mimic = pd.read_sql('select * from procedures_icd', con = mimicconn) 
obs_proc_mimic

In [ ]:
Procedures_mimic_ = obs_proc_mimic[obs_proc_mimic['seq_num'] == 1]
Procedures_mimic_

In [ ]:
d_proc = pd.read_sql('select * from d_icd_procedures', con = mimicconn)
d_proc

In [ ]:
dictPro_ = dict(zip(d_proc.icd_code, d_proc.long_title))
dictPro_

In [ ]:
Procedures_mimic_['long_title'] = Procedures_mimic_['icd_code'].map(dictPro_)
Procedures_mimic_

In [ ]:
procedure_concept = pd.read_sql('select * from `concept` where class_id = 2', con = conn) 
procedure_concept

In [ ]:
dictproc_ = dict(zip(procedure_concept.description, procedure_concept.concept_id))
dictproc_

In [ ]:
Procedures_mimic_['concept_id'] = Procedures_mimic_['long_title'].map(dictproc_)
Procedures_mimic_

In [ ]:
Procedures_mimic_ = Procedures_mimic_.rename({"subject_id": "patient_id", "hadm_id": "encounter_id", "icd_code":"diagnosis_coded_name", "concept_id":"diagnosis_coded"}, axis='columns')
Procedures_mimic_ = Procedures_mimic_.drop(columns=['seq_num', 'chartdate', 'icd_version', 'long_title'])
Procedures_mimic_['date_created']= '2022-01-01'
import uuid
Procedures_mimic_['uuid'] = [uuid.uuid4() for x in range(len(Procedures_mimic_))]
Procedures_mimic_['creator'] = 1
Procedures_mimic_

In [ ]:
column_names = ['diagnosis_coded','diagnosis_coded_name', 'encounter_id', 'patient_id', 'uuid', 'creator', 'date_created']

Procedures_mimic_ = Procedures_mimic_.reindex(columns=column_names)
Procedures_mimic_['certainty'] = 'CONFIRMED'
Procedures_mimic_

In [ ]:
encounter_diagnosis = diagnoses_mimic_.append(Procedures_mimic_)


In [ ]:
encounter_diagnosis=encounter_diagnosis.drop(columns=['diagnosis_coded_name'])
encounter_diagnosis

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in encounter_diagnosis.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in encounter_diagnosis.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `encounter_diagnosis` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()

In [ ]:
# openmrscursor.execute('Delete from encounter_diagnosis')
# conn.commit() 